In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,PowerTransformer
from scipy import stats
from sklearn.pipeline import Pipeline

In [8]:
# need to be able to import src
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# absolute path to the src directory
src_path = os.path.abspath("../src")

# Add src directory to system path
sys.path.append(src_path)

# get the /MLCBAss1/Assignment-1 path
ass_path=sys.path[-2]

print(ass_path)


/home/sdi1700187/MLCBAss1/Assignment-1


In [9]:
from functions import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# we load the two datasets
dev_set_df=pd.read_csv("../data/assignment1_dev_set.csv")
val_set_df=pd.read_csv("../data/assignment1_val_set.csv")


In [11]:
# we clean the datasets
frames_dev=clean_data(dev_set_df)
frames_val=clean_data(val_set_df)

No duplicates in the set
No duplicates in the set


In [12]:
data_path=ass_path + "/data"

# we save the two clean datasets
save_clean_data(frames=frames_dev,mode="Dev",data_path=data_path)
save_clean_data(frames=frames_val,mode="Eval",data_path=data_path)

Creating datframe for the clean data...
The shape of the clean dataframe is: (489, 137)
Saving final clean data to: /home/sdi1700187/MLCBAss1/Assignment-1/data/development_final_data.csv
Saved the clean dataframe at: /home/sdi1700187/MLCBAss1/Assignment-1/data/development_final_data.csv
Creating datframe for the clean data...
The shape of the clean dataframe is: (211, 137)
Saving final clean data to: /home/sdi1700187/MLCBAss1/Assignment-1/data/evaluation_final_data.csv
Saved the clean dataframe at: /home/sdi1700187/MLCBAss1/Assignment-1/data/evaluation_final_data.csv
